# Collect Deletion Ratios for each MT engine

## Step 1:
Create sub table of machine translation engine and associated revision ids in personal directory. 

Note: This was done because the deletion ratio queries below require a list of revision ids associated with each mt service (this data is available within a diffrent database (mariadb) so direct join within the query was not feasible and looping through list resulted in long query run times)

The query I used to create the sub table is documented below. This was inputted directly into hive client on stat8. For the data, I just copied over the mt_service_compare_ids.csv table I generated in the analysis but a query could be used to populate the table as well if preferred

In [ ]:
# Run directly Hive
# DROP TABLE IF EXISTS mneisler.cx_revision_ids_by_mtservice;

# CREATE EXTERNAL TABLE IF NOT EXISTS mneisler.cx_revision_ids_by_mtservice(
#   `target_revision_id`     double   COMMENT 'mediawiki revision id',
#   `mt_service`              string   COMMENT 'machine translation service engine'

# )
# row format delimited fields terminated by ','
# STORED AS TEXTFILE
# LOCATION '/user/mneisler/cx_revision_ids_by_mtservice'
# tblproperties('skip.header.line.count'='1')
# ;

# -- use hive to manually load revision ids and mt service data set
# LOAD DATA LOCAL INPATH 'mt_service_compare_ids.csv' OVERWRITE INTO TABLE mneisler.cx_revision_ids_by_mtservice;


## Elia Deletion Ratios

In [ ]:

query = """
-- find both cx and non-cx created articles 
WITH created_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS created_cx_total
FROM wmf.mediawiki_history mwh
-- limit to only cx revision ids
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
    snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
-- review articles published  in February 2022
    AND event_timestamp >= '2022-02-01'
    AND event_entity = 'revision'
    AND event_type = 'create' 
-- identified as Elia
    AND mtc.mt_service = 'Elia'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
GROUP BY  
  wiki_db
),

--find all deleted articles that were created with cx 

deleted_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS deleted_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
       snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
-- find revisions moved to the archive table
    AND event_type = 'create'
    AND event_timestamp >= '2022-02-01'
    AND revision_is_deleted_by_page_deletion = TRUE
-- identified as Elia
    AND mtc.mt_service = 'Elia'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
-- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
GROUP BY  
  wiki_db
)

-- main query to aggregate and join sources above
SELECT
    created_articles.wiki,
    created_cx_total,
    deleted_cx_total
FROM created_articles
LEFT JOIN deleted_articles ON 
    created_articles.wiki = deleted_articles.wiki
"""

elia_deletion_data = spark.run(query)

In [ ]:
#save data to csv
elia_deletion_data.to_csv('elia_deletion_data.csv', index = False)

## Yandex

In [ ]:
#obtain deletion ratio for all articles on target language wikis since FLores was deployed
query = """
-- find both cx and non-cx created articles 
WITH created_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS created_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
    snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
    AND event_type = 'create' 
    AND event_timestamp >= '2022-02-01'
-- identified as Elia
    AND mtc.mt_service = 'Yandex'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
GROUP BY  
  wiki_db
),

--find all deleted articles that were created with cx 

deleted_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS deleted_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
       snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
    AND event_timestamp >= '2022-02-01'
-- find revisions moved to the archive table
    AND event_type = 'create'
    AND revision_is_deleted_by_page_deletion = TRUE
-- identified as Yandex
    AND mtc.mt_service = 'Yandex'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
-- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
GROUP BY  
  wiki_db
)

-- main query to aggregate and join sources above
SELECT
    created_articles.wiki,
    created_cx_total,
    deleted_cx_total
FROM created_articles
LEFT JOIN deleted_articles ON 
    created_articles.wiki = deleted_articles.wiki
"""

In [ ]:
yandex_deletion_data = spark.run(query)

In [ ]:
#save data to csv
yandex_deletion_data.to_csv('yandex_deletion_data.csv', index = False)

## OpusMT

In [ ]:
#obtain deletion ratio for all articles on target language wikis since FLores was deployed
query = """
-- find both cx and non-cx created articles 
WITH created_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS created_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
    snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
    AND event_type = 'create' 
    AND event_timestamp >= '2022-02-01'
-- identified as OpusMT
    AND mtc.mt_service = 'OpusMT'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
GROUP BY  
  wiki_db
),

--find all deleted articles that were created with cx 

deleted_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS deleted_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
       snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
-- find revisions moved to the archive table
    AND event_type = 'create'
    AND event_timestamp >= '2022-02-01'
    AND revision_is_deleted_by_page_deletion = TRUE
-- identified as OpusmT
    AND mtc.mt_service = 'OpusMT'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
-- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
GROUP BY  
  wiki_db
)

-- main query to aggregate and join sources above
SELECT
    created_articles.wiki,
    created_cx_total,
    deleted_cx_total
FROM created_articles
LEFT JOIN deleted_articles ON 
    created_articles.wiki = deleted_articles.wiki
"""

In [ ]:
opus_deletion_data = spark.run(query)

In [ ]:
#save data to csv
opus_deletion_data.to_csv('opus_deletion_data.csv', index = False)

## LingoCloud

In [ ]:
#obtain deletion ratio for all articles on target language wikis since FLores was deployed
query = """
-- find both cx and non-cx created articles 
WITH created_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS created_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
    snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
    AND event_type = 'create' 
     AND event_timestamp >= '2022-02-01'
-- identified as OpusmT
    AND mtc.mt_service = 'LingoCloud'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
GROUP BY  
  wiki_db
),

--find all deleted articles that were created with cx 

deleted_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS deleted_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
       snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
-- find revisions moved to the archive table
    AND event_type = 'create'
     AND event_timestamp >= '2022-02-01'
    AND revision_is_deleted_by_page_deletion = TRUE
-- identified as LingoCloud
    AND mtc.mt_service = 'LingoCloud'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
-- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
GROUP BY  
  wiki_db
)

-- main query to aggregate and join sources above
SELECT
    created_articles.wiki,
    created_cx_total,
    deleted_cx_total
FROM created_articles
LEFT JOIN deleted_articles ON 
    created_articles.wiki = deleted_articles.wiki
"""

In [ ]:
lingocloud_deletion_data = spark.run(query)

In [ ]:
#save data to csv
lingocloud_deletion_data.to_csv('lingocloud_deletion_data.csv', index = False)

## Apertium

In [ ]:
#obtain deletion ratio for all articles on target language wikis since FLores was deployed
query = """
-- find both cx and non-cx created articles 
WITH created_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS created_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
    snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
    AND event_type = 'create' 
      AND event_timestamp >= '2022-02-01'
-- identified as Apertium
    AND mtc.mt_service = 'Apertium'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
GROUP BY  
  wiki_db
),

--find all deleted articles that were created with cx 

deleted_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS deleted_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
       snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
-- find revisions moved to the archive table
    AND event_type = 'create'
      AND event_timestamp >= '2022-02-01'
    AND revision_is_deleted_by_page_deletion = TRUE
-- identified as Apertium
    AND mtc.mt_service = 'Apertium'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
-- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
GROUP BY  
  wiki_db
)

-- main query to aggregate and join sources above
SELECT
    created_articles.wiki,
    created_cx_total,
    deleted_cx_total
FROM created_articles
LEFT JOIN deleted_articles ON 
    created_articles.wiki = deleted_articles.wiki
"""

In [ ]:
apertium_deletion_data = spark.run(query)

In [ ]:
#save data to csv
apertium_deletion_data.to_csv('apertium_deletion_data.csv', index = False)

## Google

In [ ]:
query = """
-- find both cx and non-cx created articles 
WITH created_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS created_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
    snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
    AND event_type = 'create' 
-- identified as Google
    AND mtc.mt_service = 'Google'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
GROUP BY  
  wiki_db
),

--find all deleted articles that were created with cx 

deleted_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS deleted_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
       snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
-- find revisions moved to the archive table
    AND event_type = 'create'
    AND revision_is_deleted_by_page_deletion = TRUE
-- identified as Google
    AND mtc.mt_service = 'Google'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
-- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
GROUP BY  
  wiki_db
)

-- main query to aggregate and join sources above
SELECT
    created_articles.wiki,
    created_cx_total,
    deleted_cx_total
FROM created_articles
LEFT JOIN deleted_articles ON 
    created_articles.wiki = deleted_articles.wiki
"""

In [ ]:
google_deletion_data = spark.run(query)

In [ ]:
#save data to csv
google_deletion_data.to_csv('google_deletion_data.csv', index = False)

## Flores

In [ ]:
#obtain deletion ratio for all articles on target language wikis since FLores was deployed
query = """
-- find both cx and non-cx created articles 
WITH created_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS created_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
    snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
    AND event_type = 'create' 
      AND event_timestamp >= '2022-02-01'
-- identified as NLLB-200
    AND mtc.mt_service = 'NLLB-200'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
GROUP BY  
  wiki_db
),

--find all deleted articles that were created with cx 

deleted_articles AS (

SELECT
    wiki_db AS wiki,
    COUNT(*) AS deleted_cx_total
FROM wmf.mediawiki_history mwh
JOIN mneisler.cx_revision_ids_by_mtservice mtc
ON mwh.revision_id = mtc.target_revision_id
WHERE
       snapshot = '2022-09'
-- only look at new page creations
    AND revision_parent_id = 0
    AND event_entity = 'revision'
-- find revisions moved to the archive table
    AND event_type = 'create'
    AND revision_is_deleted_by_page_deletion = TRUE
-- identified as NLLB-200
    AND mtc.mt_service = 'NLLB-200'
      AND event_timestamp >= '2022-02-01'
    AND ARRAY_CONTAINS(revision_tags, 'contenttranslation')
-- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
GROUP BY  
  wiki_db
)

-- main query to aggregate and join sources above
SELECT
    created_articles.wiki,
    created_cx_total,
    deleted_cx_total
FROM created_articles
LEFT JOIN deleted_articles ON 
    created_articles.wiki = deleted_articles.wiki
"""

In [ ]:
flores_deletion_data = spark.run(query)

In [ ]:
#save data to csv
flores_deletion_data.to_csv('flores_deletion_data.csv', index = False)